# 基因演算法範例 - 股票投資分析

此範例展示如何使用基因演算法挑選最適合的投資標的。範例中選用了三支台股股票，並基於平均日報酬率與風險(標準差)來評估適應度，最終目標是選擇最佳的投資標的。

## 參數與資料設定
在這部分，我們設定股票代碼、資料範圍，以及基因演算法的參數（族群大小、世代數等）。

In [13]:
import yfinance as yf
import numpy as np
import random

# 股票代碼
stocks = ["2330.TW", "2303.TW", "2317.TW", "3008.TW"]  # 台積電、聯電、鴻海、大立光
start_date = "2023-01-01"
end_date = "2024-12-31"

# GA參數
pop_size = 10  # 族群大小
generations = 10  # 演化世代數
mutation_rate = 0.1

## 抓取與處理股票資料
我們使用 `yfinance` 套件下載股票的調整收盤價，並計算每日報酬率與風險（標準差）。

In [14]:
data = {}
for stock in stocks:
    df = yf.download(stock, start=start_date, end=end_date)
    data[stock] = df['Adj Close']

# 將資料對齊 (以防有缺值狀況)
prices = np.column_stack([data[s].values for s in stocks])
# 若有NaN，簡單以前值填補
where_nan = np.isnan(prices)
prices[where_nan] = np.nanmean(prices, axis=0)[np.where(where_nan)[1]]

# 計算每日報酬率
returns = (prices[1:] - prices[:-1]) / prices[:-1]
mean_returns = np.mean(returns, axis=0)
std_returns = np.std(returns, axis=0)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


## 定義適應度函數
適應度函數的目標是最大化平均日報酬率並最小化風險。

In [15]:
def fitness(chromosome):
    # chromosome為長度3的list, 例如 [1,0,0], 表示選擇stocks[0]
    idx = chromosome.index(1)
    m = mean_returns[idx]
    s = std_returns[idx]
    fit = m - s
    return fit

## 初始化族群
每個染色體表示選擇哪一支股票，每個染色體為一個長度為3的二元列表，其中只有一個值為1，表示選定的股票。

In [16]:
def create_chromosome():
    # 隨機選1支
    ch = [0, 0, 0]
    ch[random.randint(0, 2)] = 1
    return ch

population = [create_chromosome() for _ in range(pop_size)]

## 基因演算法迭代流程
在這部分，我們進行適應度評估、選擇、交配與突變，並生成新族群。

In [17]:
for gen in range(generations):
    # 計算適應度
    fitness_values = [fitness(ch) for ch in population]

    # 印出本世代最佳
    best_fit = max(fitness_values)
    best_ch = population[fitness_values.index(best_fit)]
    print(f"Generation {gen+1}: Best Fitness = {best_fit:.6f}, Chromosome = {best_ch}")

    # 簡單菁英保存
    new_population = [best_ch]

    # 輪盤選擇函式
    def roulette_wheel(fits):
        pick = random.random() * sum(fits)
        current = 0
        for i, f in enumerate(fits):
            current += f
            if current > pick:
                return i
        return len(fits) - 1

    while len(new_population) < pop_size:
        p1_idx = roulette_wheel(fitness_values)
        p2_idx = roulette_wheel(fitness_values)
        parent1 = population[p1_idx]
        parent2 = population[p2_idx]

        # 單點交配
        cross_point = random.randint(1, 2)
        child1 = parent1[:cross_point] + parent2[cross_point:]
        child2 = parent2[:cross_point] + parent1[cross_point:]

        def fix_chromosome(ch):
            if sum(ch) == 1:
                return ch
            new_ch = [0, 0, 0]
            new_ch[random.randint(0, 2)] = 1
            return new_ch

        child1 = fix_chromosome(child1)
        child2 = fix_chromosome(child2)

        def mutate(ch):
            if random.random() < mutation_rate:
                m_ch = [0, 0, 0]
                m_ch[random.randint(0, 2)] = 1
                return m_ch
            return ch

        child1 = mutate(child1)
        child2 = mutate(child2)

        new_population.append(child1)
        if len(new_population) < pop_size:
            new_population.append(child2)

    population = new_population

Generation 1: Best Fitness = -0.014404, Chromosome = [0, 1, 0]
Generation 2: Best Fitness = -0.014404, Chromosome = [0, 1, 0]
Generation 3: Best Fitness = -0.014404, Chromosome = [0, 1, 0]
Generation 4: Best Fitness = -0.014404, Chromosome = [0, 1, 0]
Generation 5: Best Fitness = -0.014404, Chromosome = [0, 1, 0]
Generation 6: Best Fitness = -0.014404, Chromosome = [0, 1, 0]
Generation 7: Best Fitness = -0.014404, Chromosome = [0, 1, 0]
Generation 8: Best Fitness = -0.014404, Chromosome = [0, 1, 0]
Generation 9: Best Fitness = -0.014404, Chromosome = [0, 1, 0]
Generation 10: Best Fitness = -0.014404, Chromosome = [0, 1, 0]


## 最終結果
列出每支股票的回報率、風險與適應度，並顯示最佳選擇的股票。

In [18]:
fitness_values = [fitness(ch) for ch in population]
best_fit = max(fitness_values)
best_ch = population[fitness_values.index(best_fit)]
best_stock = stocks[best_ch.index(1)]
final_idx = best_ch.index(1)
final_return = mean_returns[final_idx]
final_risk = std_returns[final_idx]

for i, stock in enumerate(stocks):
    print(f"{stock}: 平均回報率 = {mean_returns[i]:.6f}, 風險 = {std_returns[i]:.6f}, 適應度 = {mean_returns[i] - std_returns[i]:.6f}")

print("===== 最終結果 =====")
print(f"股票選擇：{stocks}")
print(f"最佳投資組合選擇的股票: {best_stock}")
print(f"預期日平均回報率: {final_return:.6f}")
print(f"風險(報酬率標準差): {final_risk:.6f}")
print(f"適應度: {best_fit:.6f}")

2330.TW: 平均回報率 = 0.002038, 風險 = 0.018454, 適應度 = -0.016416
2303.TW: 平均回報率 = 0.000458, 風險 = 0.014862, 適應度 = -0.014404
2317.TW: 平均回報率 = 0.001728, 風險 = 0.019320, 適應度 = -0.017592
3008.TW: 平均回報率 = 0.000741, 風險 = 0.021098, 適應度 = -0.020358
===== 最終結果 =====
股票選擇：['2330.TW', '2303.TW', '2317.TW', '3008.TW']
最佳投資組合選擇的股票: 2303.TW
預期日平均回報率: 0.000458
風險(報酬率標準差): 0.014862
適應度: -0.014404
